In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import re

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13


In [2]:
#messo n_parseg = 150 e sembra avere senso lo spettrogramma anche se non sono 100 come nell'articolo
parseg = 250
neglect_t = 3000
function = "flat"
subnets = "STN"
sigm_par = float('inf')

#new_function = re.sub(r"[^0-9]", "", function) #lascia solo numero funzione

# Trovare $\alpha$

In [3]:
Dd = "0.9500"

In [4]:
#funzione Dd
data1 = np.loadtxt("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0/ext_rateD2.txt").T
#spikesim
s1 = utils.SpikeSim("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0", 'new_sim_parallel.yaml', neglect_t, np.max(data1[0]))

In [5]:
alpha1, alpha3 = s1.threshold(pop=subnets, N_parseg=parseg)

nparseg = 250	noverlap=125


In [6]:
print(alpha1, alpha3)

1.349591348704985 0.3502194468379441


In [7]:
Dd = "0.8500"

In [8]:
#funzione Dd
data2 = np.loadtxt("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0/ext_rateD2.txt").T
#spikesim
s2 = utils.SpikeSim("./build/output/n1/" + function + "_0.00_0.00000_0.00_" + Dd + "_1.00_1.00_1.6/0", 'new_sim_parallel.yaml', neglect_t, np.max(data2[0]))

In [9]:
alpha2, alpha4 = s2.threshold(pop=subnets, N_parseg=parseg)

nparseg = 250	noverlap=125


In [10]:
print(alpha2, alpha4)

0.052611029228205916 0.02671846869324434


In [11]:
print(alpha1)
print(alpha2)
print(alpha3)
print(alpha4)

1.349591348704985
0.052611029228205916
0.3502194468379441
0.02671846869324434


In [12]:
alpha = (alpha1 + alpha2)/2
print(alpha)

0.7011011889665955


In [13]:
alpha_mean = (alpha3 + alpha4)/2
print(alpha_mean)

0.18846895776559422


# SIGMOIDPULSE k2

In [14]:
parseg = 250
overlap = 0.5

neglect_t = 3000
t_mid = 13000
t_mid2 = 23000
sigm_par = 0.002

t_tot = 33000

dd_low = 0.850

In [15]:
sigm_par_poiss = "{:.5f}".format(sigm_par)
function = "sigmoidpulse"
Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]


In [16]:
#funzione Dd
data = np.loadtxt(f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/0/ext_rateD2.txt").T
#spikesim
s = utils.SpikeSim(f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/0", 'new_sim_parallel.yaml', neglect_t, np.max(data[0]))

In [17]:
output=s.periodogram(pop=subnets, res=1., N_parseg=parseg)
f = output[0]
t = output[1]
Sxx = output[2]
data[0] = data[0]/1000
data[1] = data[1]/1.083


# faccio somma nel tempo per trovare frequenza max #######################################################
sum1 = np.sum(Sxx, axis=1) #somma per tutte frequenze

# Ottieni l'indice del massimo della funzione
max_index = np.argmax(sum1)
max_value = sum1[max_index]
print('frequenza del massimo:', f[max_index])
print('valore del massimo:', max_value)


nparseg = 250	noverlap=125
frequenza del massimo: 16.5
valore del massimo: 160.33015874960208


In [19]:
%matplotlib widget

mask = (8 <= f) & (f <= 26)
Sxx_lim = Sxx[mask, :]
# Calculate the mean power values over time
pow_t = [np.mean(Sxx_lim[:, i]) for i in range(len(t))]

# Using a rolling window to calculate the moving average
window_size = 2  # Replace with your desired window size
kernel = np.ones(window_size) / window_size

# Apply symmetric padding at the beginning and end of the pow_t array
pad_length = window_size // 2
pow_t_padded = np.pad(pow_t, (pad_length, pad_length), 'symmetric')

moving_avg = np.convolve(pow_t_padded, kernel, mode='same')

# Remove the padding from the moving average to match the original pow_t length
moving_avg = moving_avg[pad_length:-pad_length]

# Create a plot of the values over time
fig, ax1 = plt.subplots(figsize=(12, 5))

# Convert pow_t to a numpy array for boolean indexing
pow_t = np.array(pow_t)

ax1.axhline(y=alpha_mean, color='red', linestyle='-', label='threshold α')
#ax1.plot(t[pow_t >= alpha_mean], pow_t[pow_t >= alpha_mean], 'r+', markersize=10, markeredgewidth=2, label='indices belonging to burst')
ax1.fill_between(t, alpha_mean, pow_t, where=(pow_t >= alpha_mean), color='red', alpha=0.3, label='Bursts')

ax1.plot(t, pow_t, label='Average intensity')

# Add the moving average to ax1's plot
#ax1.plot(t, moving_avg, label=f'Moving Average ({window_size}-point)', color='orange')

plt.xlabel('Time [s]')
plt.ylabel('Mean β PSD')
plt.legend()

plt.title(f'Bursts in Pulse τ = {1/sigm_par/1000} [s]')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
bursts = utils.burst_sequence((pow_t>=alpha_mean)*1)
b_durations = np.array(bursts[1])
print(bursts)
print('number of burst: ',bursts[0])
print('length of every burst: ',b_durations*parseg/2000,'s')
print('Average burst duration: {:.2f} s'.format(bursts[2]*parseg/2000))
print('Burst fraction: {:.1f} %'.format(np.sum(b_durations)*100/len(pow_t)))
print('Burst frequency: {:.2f} burst/s'.format(bursts[0]/(t_tot - neglect_t)*1000))


number of burst, length of every burst, mean length: (11, [6, 4, 2, 16, 6, 11, 6, 6, 16, 2, 42], 10.636363636363637)
number of burst:  11
length of every burst:  [0.75  0.5   0.25  2.    0.75  1.375 0.75  0.75  2.    0.25  5.25 ] s
Average burst duration: 1.33 s
Burst fraction: 49.0 %
Burst frequency: 0.37 burst/s


# PULSE 

In [21]:
parseg = 250
overlap = 0.5

neglect_t = 1250 #multipli di parseg/2
t_mid = 1400
t_mid2 = 2400
sigm_par = 0.05

t_tot = 3000

dd_low = 0.850

function = "burst2"

In [22]:
sigm_par_poiss = "{:.5f}".format(sigm_par)
Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]


In [23]:
%matplotlib widget
output = []
s = []
for simulation in simulations:
    ss = utils.SpikeSim(f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/{simulation}", 'new_sim_parallel.yaml', neglect_t, 3000)
    s.append(ss)
    output_s=ss.periodogram(pop=subnets, res=1., N_parseg=parseg)
    output.append(output_s)

nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125
nparseg = 250	noverlap=125


In [24]:
data = np.loadtxt(f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/0/ext_rateD2.txt").T

data[0] = data[0]/1000
data[1] = data[1]/1.083

t_dd = data[0][(10*neglect_t):] - data[0][10*neglect_t]
dd = data[1][(10*neglect_t):] - data[1][10*neglect_t]

In [25]:
t_d = np.array(t_dd)
d = np.array(dd)

In [26]:
for i in range(1, len(output)):
    t_d = np.concatenate((t_d, t_dd[:-1] + t_dd[-1]*i))
    d = np.concatenate((d, dd[:-1]))


In [27]:
f = output[0][0]  # Presumibilmente, f è lo stesso per tutte le simulazioni
t = np.array(output[0][1])
Sxx = output[0][2]

for i in range(1, len(output)):
    t = np.concatenate((t, output[i][1] + t[-1] + parseg/2/1000))
    Sxx = np.concatenate((Sxx, output[i][2]), axis=1)

In [28]:
 %matplotlib widget
# Applica la maschera e definisci Sxx_lim e f_lim come prima
mask = (8 <= f) & (f <= 26)
Sxx_lim = Sxx[mask, :]
f_lim = f[mask]

# Calcola i valori medi della potenza nel tempo
pow_t = [np.mean(Sxx_lim[:, i]) for i in range(len(t))]
pow_t = np.array(pow_t)  # Converti in un array NumPy

# Crea una figura e tre assi
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(11, 13), sharex=True)

# Primo plot: Contour plot di Sxx_lim
ax1.contourf(t, f_lim, Sxx_lim)
ax1.set_ylabel('Frequency [Hz]')
ax1.set_title(f'Sequence of β bursts')
ax1.text(0.01, 0.95, '(a)', transform=ax1.transAxes, fontsize=16, fontweight='bold', va='top')

# Secondo plot: PSD media sopra alpha_mean
ax2.axhline(y=alpha_mean, color='red', linestyle='-', label='threshold α')
#ax2.plot(t[pow_t >= alpha_mean], pow_t[pow_t >= alpha_mean], 'r+', markersize=10, markeredgewidth=2, label='indices belonging to burst')
ax2.fill_between(t, alpha_mean, pow_t, where=(pow_t >= alpha_mean), color='red', alpha=0.3, label='Bursts')
ax2.plot(t, pow_t, label='Mean β PSD')
ax2.set_ylabel('Mean β PSD')
ax2.text(0.01, 0.8, '(b)', transform=ax2.transAxes, fontsize=16, fontweight='bold', va='top')
ax2.legend()

# Terzo plot: La tua terza serie di dati
ax3.plot(t_d, d*10, color = 'red')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Dopamine depletion y [Dd]')  # Sostituisci 'D' con un'etichetta appropriata per i tuoi dati
ax3.text(0.01, 0.95, '(c)', transform=ax3.transAxes, fontsize=16, fontweight='bold', va='top')

# Mostra la figura
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
bursts = utils.burst_sequence((pow_t>=alpha_mean)*1)
b_durations = np.array(bursts[1])
print(bursts)
print('number of burst: ',bursts[0])
print('length of every burst: ',b_durations*parseg/2000,'s')
print('Average burst duration: {:.2f} s'.format(bursts[2]*parseg/2000))
print('Burst fraction: {:.1f} %'.format(np.sum(b_durations)*100/len(pow_t)))
print('Burst frequency: {:.2f} burst/s'.format(bursts[0]/(t_tot - neglect_t)/32*1000))


number of burst, length of every burst, mean length: (24, [3, 4, 4, 4, 5, 5, 5, 5, 2, 3, 4, 4, 4, 2, 6, 5, 4, 3, 2, 3, 6, 3, 4, 6], 4.0)
number of burst:  24
length of every burst:  [0.375 0.5   0.5   0.5   0.625 0.625 0.625 0.625 0.25  0.375 0.5   0.5
 0.5   0.25  0.75  0.625 0.5   0.375 0.25  0.375 0.75  0.375 0.5   0.75 ] s
Average burst duration: 0.50 s
Burst fraction: 23.1 %
Burst frequency: 0.43 burst/s


In [36]:
#patient values
print('Average burst time: ',3*parseg/2000,'s')
print('Burst fraction: {:.2f} %'.format(np.sum(3)*100*parseg/2/1700))
print('Burst frequency: {:.2f} burst/s'.format(1/(1700)*1000))

Average burst time:  0.375 s
Burst fraction: 22.06 %
Burst frequency: 0.59 burst/s
